In [70]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np
import datetime
import os
# 忽略掉警告信息，使用界面更清爽
import warnings 
warnings.filterwarnings("ignore")

print(tf.__version__)
print(np.__version__)

2.2.0
1.18.4


In [71]:
# 在终端执行程序时指定GPU  

# CUDA_VISIBLE_DEVICES=1   python  your_file.py

# 这样在跑你的网络之前，告诉程序只能看到1号GPU，其他的GPU它不可见

# 可用的形式如下：

# CUDA_VISIBLE_DEVICES=1           Only device 1 will be seen
# CUDA_VISIBLE_DEVICES=0,1         Devices 0 and 1 will be visible
# CUDA_VISIBLE_DEVICES="0,1"       Same as above, quotation marks are optional
# CUDA_VISIBLE_DEVICES=0,2,3       Devices 0, 2, 3 will be visible; device 1 is masked
# CUDA_VISIBLE_DEVICES=""          No GPU will be visible



# 在Python代码中指定GPU

# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"



# 设置定量的GPU使用量

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.9 # 占用GPU90%的显存
# session = tf.Session(config=config)



# 设置最小的GPU使用量

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.Session(config=config)

In [72]:
# 指定CUDA设备
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [73]:
# tf.test.is_gpu_available() # 高版本已废弃不再使用
tf.config.list_physical_devices('GPU')

[]

In [74]:
# gpus = tf.config.list_physical_devices(device_type='GPU')
# cpus = tf.config.list_physical_devices(device_type='CPU')
# print(gpus, cpus)

In [75]:
# GPU
# gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
# print(os.environ['CUDA_VISIBLE_DEVICES'])
# CPU
# cpus = tf.config.experimental.list_physical_devices(device_type='CPU')

In [76]:
# tf.config.list_physical_devices('GPU')

In [77]:
# 划分训练数据，测试数据
mnist = np.load("mnist.npz")
x_train, y_train, x_test, y_test = mnist['x_train'],mnist['y_train'],mnist['x_test'],mnist['y_test']
# 归一化
x_train, x_test = x_train / 255.0, x_test / 255.0


In [78]:
# 增加通道维度
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# x_train
# x_test

In [79]:
# tf.data.Dataset.from_tensor_slices
# 该函数是dataset核心函数之一，它的作用是把给定的元组、列表和张量等数据进行特征切片。切片的范围是从最外层维度开始的。
# 如果有多个特征进行组合，那么一次切片是把每个组合的最外维度的数据切开，分成一组一组的。
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [80]:
class MyModel(Model):
    def __init__(self,**kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation = 'relu')
        self.d2 = Dense(10, activation='softmax')
        
    @tf.function
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [81]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [82]:
# @tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)
    
# @tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [83]:
model = MyModel()

In [84]:
stamp = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
import os
logdir = os.path.join('keras_logv4')

summary_writer = tf.summary.create_file_writer(logdir)

EPOCHS = 1

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_ds:
        train_step(x_train, y_train)
    with summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
        
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()
    
with summary_writer.as_default():
    tf.summary.trace_on(graph=True, profiler=False)
    tf.summary.trace_export(name="model_trace", step=3, profiler_outdir=None)
    


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [85]:
# !rm -rf ./keras_logv4/*

In [88]:
summary_writer = tf.summary.create_file_writer('./tensorboard')
log_dir = "graph"

In [89]:
summary_writer = tf.summary.create_file_writer('./tensorboard')

train_summary_writer = tf.summary.create_file_writer('./tensorboard')

EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_ds:
        train_step(x_train, y_train)
        
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
    
    for(x_test, y_test) in test_ds:
        test_step(x_test, y_test)
        
    tf.summary.trace_on(graph=True, profiler=True)
    
    with summary_writer.as_default():
        tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=log_dir)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, TestLoss : {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                         train_loss.result(),
                         train_accuracy.result() * 100,
                         test_loss.result(),
                         test_accuracy.result() * 100))
    
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.03226933255791664, Accuracy: 98.98333740234375, TestLoss : 0.048861049115657806, Test Accuracy: 98.44999694824219
Epoch 2, Loss: 0.013172667473554611, Accuracy: 99.56500244140625, TestLoss : 0.06103254109621048, Test Accuracy: 98.29999542236328
Epoch 3, Loss: 0.009416600689291954, Accuracy: 99.67832946777344, TestLoss : 0.05992802977561951, Test Accuracy: 98.40999603271484
Epoch 4, Loss: 0.00683900760486722, Accuracy: 99.77166748046875, TestLoss : 0.06621938198804855, Test Accuracy: 98.32999420166016
Epoch 5, Loss: 0.006631186231970787, Accuracy: 99.75833129882812, TestLoss : 0.062243957072496414, Test Accuracy: 98.47999572753906
